# Heatmaps for carbon intensity in 2020

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join
import logging
import json

import pandas as pd
import gridemissions
from gridemissions import api
from gridemissions.load import BaData
from gridemissions import viz


In [ ]:
savefigs = False

In [ ]:
COLORS, PAGE_WIDTH, ROW_HEIGHT = viz.set_plots()

In [ ]:
file_name = join(gridemissions.config["APEN_PATH"], "data", "EBA_%s.csv")
co2 = BaData(fileNm=file_name % "co2", variable="CO2")
elec = BaData(fileNm=file_name % "elec", variable="E")

In [ ]:
start = pd.to_datetime("20200101T0000Z")
end = pd.to_datetime("20210101T0000Z")

In [ ]:
ba = "CISO"
co2i = pd.DataFrame({ba: (
    co2.df.loc[start:end, co2.get_cols(ba, field="D")].values.flatten()
    / elec.df.loc[start:end, elec.get_cols(ba, field="D")].values.flatten())
 for ba in co2.regions
}, index=co2.df.loc[start:end].index)

# Store index before shifting
utc_idx = co2i.index
co2i.index -= pd.Timedelta("6h")

In [ ]:
f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5*ROW_HEIGHT))
viz.heatmap(co2i["CISO"], fax=(f, ax), cmap="RdYlGn_r", cbar_label="CISO - kg/MWh", transpose=True)
viz.add_watermark(ax)
f.tight_layout()
if savefigs:
    f.savefig(join(gridemissions.config["APEN_PATH"], "CISO_in_2020_A.pdf"))


In [ ]:
n=40
bas = co2.df.loc[start:end, co2.get_cols(field="D")].sum().sort_values(ascending=False).index[0:n]
bas = list(map(lambda x: x.split("_")[1], bas))
bas

In [ ]:
f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5*ROW_HEIGHT))
viz.heatmap(co2i["CISO"], fax=(f, ax), vmin=100, vmax=900, cmap="RdYlGn_r", cbar_label="CISO - kg/MWh", transpose=True)
viz.add_watermark(ax)
f.tight_layout()
if savefigs:
    f.savefig(join(gridemissions.config["APEN_PATH"], "CISO_in_2020_B.pdf"))

In [ ]:
n = 40
bas = co2.df.loc[start:end, co2.get_cols(field="D")].sum().sort_values(ascending=False).index[0:n]
bas = list(map(lambda x: x.split("_")[1], bas))

nrows = n // 4

f, ax = plt.subplots(nrows, 4, figsize=(1.2*PAGE_WIDTH, nrows/2* ROW_HEIGHT))
ax = ax.flatten()

for iba, ba in enumerate(bas[:-1]):
    ax[iba].set_title(ba)
    viz.heatmap(co2i[ba], fax=(f, ax[iba]), vmin=100, vmax=900, cmap="RdYlGn_r", cbar_label="kg/MWh", with_cbar=False)


for a in ax:
    a.set_yticks([])
    a.set_xticks([])
    a.set_ylabel("")
    a.set_xlabel("")
f.tight_layout()

ba = bas[-1]
iba = len(bas)-1
ax[iba].set_title(ba)
viz.heatmap(co2i[ba], fax=(f, ax[iba]), vmin=100, vmax=900, cmap="RdYlGn_r", cbar_label="kg/MWh", with_cbar=True, cbar_ax=[ax[-4:]])
for a in ax:
    a.set_yticks([])
    a.set_xticks([])
    a.set_ylabel("")
    a.set_xlabel("")
viz.add_watermark(ax[iba], y=-.05)

if savefigs:
    f.savefig(join(gridemissions.config["APEN_PATH"], "top_40_in_2020.pdf"))

# From 2019 to 2021

In [ ]:
from io import StringIO

elec2 = []
for start, end in [("20190101", "20191231"), ("20191231", "20201231"), ("20201231", "20211231")]:
    data = api.retrieve(variable="elec", ba=bas, start=start, end=end, field="D")
    elec2.append(pd.read_csv(StringIO(data), index_col=0))
elec2 = pd.concat(elec2)

In [ ]:
co22 = []
for start, end in [("20190101", "20191231"), ("20191231", "20201231"), ("20201231", "20211231")]:
    data = api.retrieve(variable="co2", ba=bas, start=start, end=end, field="D")
    co22.append(pd.read_csv(StringIO(data), index_col=0))
co22 = pd.concat(co22)

In [ ]:
co22.index = pd.to_datetime(co22.index)
elec2.index = pd.to_datetime(elec2.index)

In [ ]:
start = pd.to_datetime("20190101T0000Z")
end = pd.to_datetime("20211231T0000Z")

co2i2 = pd.DataFrame({ba: (
    co22.loc[start:end, co2.get_cols(ba, field="D")].values.flatten()
    / elec2.loc[start:end, elec.get_cols(ba, field="D")].values.flatten())
 for ba in bas
}, index=co22.loc[start:end].index)

# Store index before shifting
utc_idx = co2i2.index
co2i2.index -= pd.Timedelta("6h")

# Drop duplicates
co2i2 = co2i2[~co2i2.index.duplicated()]

In [ ]:
f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5*ROW_HEIGHT))
plotHeatmap(co2i2, "CISO", fig=f, ax=ax, cmap="RdYlGn_r", cbar_label="CISO - kg/MWh")
f.tight_layout()
f.savefig(join(gridemissions.config["APEN_PATH"], "CISO_2019_2021_A.pdf"))

f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5*ROW_HEIGHT))
plotHeatmap(co2i2, "CISO", fig=f, ax=ax, cmap="RdYlGn_r", vmin=100, vmax=900, cbar_label="CISO - kg/MWh")
f.tight_layout()
f.savefig(join(gridemissions.config["APEN_PATH"], "CISO_2019_2021_B.pdf"))

In [ ]:
f, ax = plt.subplots(nrows, 4, figsize=(1.2*PAGE_WIDTH, nrows/2* ROW_HEIGHT))
ax = ax.flatten()

for iba, ba in enumerate(bas):
    ax[iba].set_title(ba)
    with_cbar = False
    if iba == len(ax)-2:
        with_cbar = True
    plotHeatmap(co2i2, ba, fig=f, ax=ax[iba], cmap="RdYlGn_r", vmin=100, vmax=900, with_cbar=with_cbar, cbar_label="kg/MWh")


for a in ax:
    a.set_yticks([])
    a.set_xticks([])
    a.set_ylabel("")
    a.set_xlabel("")
    
f.tight_layout()
f.savefig(join(gridemissions.config["APEN_PATH"], "top_40_in_2019_2021.pdf"))